## Prerequisites


Before using this tutorial, ensure that the following are on your system:
- <b>StegBench is installed.</b> Install via make install
- <b>Steganography Tools are on your system</b>. Installed locally or installed via bin/install_assets.sh
- <b>StegDetect Tools are on your system</b>. Installed locally or installed via bin/intall_assets.sh

We have provided a script at bin/install_assets.sh. This script will install known steganography and steganalysis tools. It will help you get started more quickly. You can run this with: 

```
bash bin/install_assets.sh
```

It is suggested that if you are using machine-learning based tools like SteganoGAN or StegDetect that you use:
- <b>CUDA enabled device.</b> Cloud computing resources can be used to access these devices


### StegBench File Sysstem 

StegBench depends on file-system assets to be present. This allows StegBench to keep track of datasets and current algorithms for image-hiding and image-detection that have been loaded into the system. 

Initialize will create all the necessary assets in the current working directory. <b>Remember that you must initialize stegbench in the directory that you wish to use it in</b>.

The file system for StegBench looks like the following:

```
stegbench_assets/
├── db/ 
|   ├── datasets/
│   └── metadata/
|   ├── source.csv
│   └── embedded.csv
├── embeddor <b>Contains all embeddor-related assets</b>
|   ├── sets
│   └── assets
|   ├── embeddors.csv
├── detector <b>Contains all detector-related assets</b>
|   ├── sets
│   └── assets
|   ├── detectors.csv
```

##### Database File Structure

The database directory (db) contains all database-related assets and images. It is composed of the following:

- <b>datasets:</b> Contains all the raw image files when they are processed/downloaded.
- <b>metadata:</b> Contains metadata about the image files that are downloaded 
- <b>source.csv</b> Contains information on the source databases
- <b>embedded.csv</b> Contains information on the steganographic databases 

##### Embeddor File Structure

The embeddor directory (embeddor) contains all embeddor-related assets. It is composed of the following: 

- <b>sets:</b> Contains information on embeddor sets (covered in later part of tutorial)
- <b>assets:</b> Contains assets needed for embedding
- <b>embeddors.csv</b> Contains information on embeddors in system 

##### Detector File Structure

The detector directory (detectr) contains all detector-related assets. It is composed of the following:

- <b>sets:</b> Contains information on detector sets (covered in laber part of tutorial)
- <b>assets:</b> Contaiins assets needed for detection 
- <b>detectors.csv</b> Contains informatin on detectors in system 


### UUID

In our system, all assets are kept track with a UUID, or in other words, a unique identifier. A UUID is assigned to databases, embeddors, and detectors. We will require UUIDs in the next few steps. The information command lets us find out the UUID for each of these assets so that we can properly use them in our code. 

An example UUID is: e1d5d46e-6763-4083-a88f-1c2d3f02776e. They follow standard UUIDv4 specifications. 

In [9]:
import stegbench as stegbench

## Step 1. Initializing StegBench

To initialize this file structure, we call the following command: 

```python
stegbench.initialize()
```

This initializes the stegbench_asset directory desribed above in your current working directory. 

In [10]:
stegbench.initialize()

initializing fs at /Users/anene/Documents/MEng/Steganography/Evaluator/stegbench/notebooks
cleaning fs...
directory does not exist
initializing directories...
initializing all files...


## Step 2. Loading the Data

### 2A: Downloading Data from Pre-Specified Sources 

We provide a set of routines to download from. There are several options for these routines available to you. They are listed in the parameter description below. 

```python
stegbench.download(name_of_routine, name_of_database)
```

- <b>name_of_routine</b>(str): The options are 'ALASKA', 'BOSS', 'BOWS2', 'COCO_Val', 'COCO_Test', 'COCO_Train', 'DIV2K_VALID', 'DIV2K_TRAIN' 
- <b>name_of_database</b>(str): Specify a user-friendly name for the database 

In [ ]:
COCO_database_uuid = stegbench.download('COCO_Val', 'COCO_Val')

### 2B: Loading a Custom Dataset

We also provide the option to process a custom dataset into the system. This dataset must already be downloaded onto your machine. To process this dataset, you call the function below with a path to the directory as well as a user-supplied name. 

```python
stegbench.process(path_to_data_directory, name_of_database)
```

- <b>path_to_data_directory</b>(str): This is the path to the directory that contain the images.
- <b>name_of_database</b>(str): Specify a user-friendly name for the database 

In [11]:
TOY_database_uuid = stegbench.process('example_dataset/', 'Toy Dataset')

applying following operations: dict_keys([])
The UUID of the dataset(s) you have processed is: 730515e3-572f-4d79-a53f-8049961aae0f


## Step 3. Loading Configuration Files 

StegBench works by processing configuration files that contain information about each of the algorithms in the system. This allows StegBench to be a highly interoperable and modular system that can easily integrate into existing steganographic pipelines. 

<b>PLEASE NOTE THAT YOU WILL HAVE TO MODIFY THE CONFIGURATION FILES TO MATCH THE TOOLS AVAILABLE ON YOUR MACHINE. IF YOU DO NOT MODIFY THEM, THE SYSTEM WILL NOT PROPERLY BE ABLE TO USE YOUR TOOLS. DIRECTIONS AVAILABLE AT [CONFIGURATION.md](../CONFIGURATION.md) ON HOW TO CREATE A CONFIGURATION.</b>

### 3A: Loading Embeddor Configuration 

<b>PLEASE READ [CONFIGURATION.md](../CONFIGURATION.md) LOCATED IN THE TOP-LEVEL DIRECTORY FOR A MORE COMPREHENSIVE REVIEW</b>

These configurations are stored in .ini files located on your machine. To load a configuration, we use the function:

```python
stegbench.add_config(config=[config_file_paths], directory=[config_directory_paths])
```

This function takes two parameters that are:

- <b>config</b>(list). List of paths to configuration files. 
- <b>directory</b>(list): List of paths to directories that contain configuration files. Useful for batch processing. 

Let us load an embeddor configuration file now. 

In [12]:
stegbench.add_config(config=['../examples/configs/embeddor/spatial-based.ini'])

starting processing of config file: /Users/anene/Documents/MEng/Steganography/Evaluator/stegbench/examples/configs/embeddor/spatial-based.ini
processing of file complete


### 3B: Loading Detector Configuration

<b>PLEASE READ [CONFIGURATION.md](../CONFIGURATION.md) LOCATED IN THE TOP-LEVEL DIRECTORY FOR A MORE COMPREHENSIVE REVIEW</b>

To load a detector configuration, we use the same function as before. In this case, let us load configurations from two different directories that contain configuration files. Please note that detectors come in two varieties: 

- <b>Classification Detectors</b>: These output a yes/no decision on if an image is steganographic or not.
- <b>Probabilistic Detectors</b>: These output a probability that an image is steganographic or not. 


In [13]:
stegbench.add_config(directory=['../examples/configs/detector_csfc/', '../examples/configs/detector_prob/'])

processing config directory: ../examples/configs/detector_csfc/
starting processing of config file: /Users/anene/Documents/MEng/Steganography/Evaluator/stegbench/examples/configs/detector_csfc/alethia.ini
processing of file complete
starting processing of config file: /Users/anene/Documents/MEng/Steganography/Evaluator/stegbench/examples/configs/detector_csfc/brute-based.ini
processing of file complete
starting processing of config file: /Users/anene/Documents/MEng/Steganography/Evaluator/stegbench/examples/configs/detector_csfc/dl-based.ini
processing of file complete
starting processing of config file: /Users/anene/Documents/MEng/Steganography/Evaluator/stegbench/examples/configs/detector_csfc/srm-based.ini
processing of file complete
processing of directory complete
processing config directory: ../examples/configs/detector_prob/
starting processing of config file: /Users/anene/Documents/MEng/Steganography/Evaluator/stegbench/examples/configs/detector_prob/dl-based.ini
processing of 

## Step 4. Retrieving System Information

Now that we have processed enough data, we would like to see information about what is currently in our system. To do this, stegbench provides the following command

```python
stegbench.info(all=True, database=True, embeddor=True, detector=True)
```

The parameters to this function are the following:

- <b>all</b>(bool): If true, all system information will be printed.
- <b>database</b>(bool): If true, all database information will be printed. 
- <b>embeddor</b>(bool): If true, all embeddor information will be printed.
- <b>detector</b>(bool): If true, all detector information will be printed. 

You shoud use this command to manually retrieve the UUID of the asset that you wish to choose in future operations. 

In [15]:
stegbench.info(all=True, database=True, embeddor=True, detector=True)

Listing all requested information....
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
All database information
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Source Databases processed: (1)
	Toy Dataset
		UUID: 730515e3-572f-4d79-a53f-8049961aae0f
		Directory path: /Users/anene/Documents/MEng/Steganography/Evaluator/stegbench/notebooks/example_dataset
		Image Count: 10
		Image Types: ['png']
----------------------------------------------------------------------------------------------------
Steganographic Databases processed: (0)
----------------------------------------------------------------------------------------------------
------------------------------------------------------

## Step 5. Creating Algorithm Sets

StegBench uses sets of tools and then applies these tools on a set of data. This allows StegBench to easily generate huge amounts of data with a variety of tools. Sets are composed of one or more tools. They are also assigned a unique identifier (UUID). You can retrieve a UUID from the info command as well. The set abstraction is very useful since it lets you apply one or more tools in a very modular fashion to a variety of databases. 

### 5A: Creating an Embeddor Set

To create an embeddor set, we have provided the following function: 

```python
stegbench.add_embeddor(embeddor_uuids, set_uuid)
```

This function will take in a list of embeddors and combine them to create an embeddor set. An embeddor set is essentially a UUID that represents a collection of one or more embeddors. The function takes the following parameters:
- <b>embeddor_uuids</b>(list): This is a list of embeddor UUIDs that will be combined into a set. The UUIDs are retrieved via the info command.  
- <b>set_uuid</b>(str): If this parameter is provided, the embeddors will be added to a pre-existing set. By default, it is set to None

In [16]:
LSBSteg_tool_uuid = '76e9535b-eabd-4aa6-8adf-5edfc1bc8f2b' #This is retrieved from the info command
SteganoLSB_tool_uuid = 'eb661d7e-a7c1-4db4-b754-70b70f1004ee' #This is retrived from the info command

embeddor_uuids = [LSBSteg_tool_uuid, SteganoLSB_tool_uuid]

embeddor_set_uuid = stegbench.add_embeddor(embeddor_uuids)

Adding embeddors: ['76e9535b-eabd-4aa6-8adf-5edfc1bc8f2b', 'eb661d7e-a7c1-4db4-b754-70b70f1004ee']
Added embeddor successfully
The new UUID of the set you have created is: 23fea9ab-7716-4d35-aa77-8dadf46f5dcd


### 5B: Creating a Detector Set

To create a detector set, we have provided the following function: 

```python
stegbench.add_detector(detector_uuid, set_uuid)
```

This function will take in a list of detectors and combine them to create a detector set. A detector set is essentially a UUID that represents a collection of one or more detectors. The function takes the following parameters:
- <b>detector_uuids</b>(list): This is a list of detector UUIDs that will be combined into a set. The UUIDs are retrieved via the info command. 
- <b>set_uuid</b>(str): If this parameter is provided, the detectors will be added to a pre-existing set. By default, it is set to None.

In [17]:
StegExpose_tool_uuid = '3a55998c-e57b-4033-bfe8-565cfc97b17f' #This is retrieved from the info command
YeNet_tool_uuid = '85f234bb-fddc-4768-aff8-2ff326f0943e' #This is retrieved from the info command

detector_uuids = [StegExpose_tool_uuid, YeNet_tool_uuid]

detector_set_uuid = stegbench.add_detector(detector_uuids)

Adding detectors: ['3a55998c-e57b-4033-bfe8-565cfc97b17f', '85f234bb-fddc-4768-aff8-2ff326f0943e']
Added detector successfully
The new UUID of the set you have created is: 3b177b6b-66a3-4012-aec1-d69ae094a18d


## Step 6. Steganographic Database Generation

To generate a steganographic database, we have provided the following function:

```python
stegbench.embed(embeddor_set_uuid, database_uuid, ratio, name)
```

The embed function takes the database_uuid to retrieve a list of database images. It then takes the embeddor_set_uuid to retrieve a list of embedding tools. Using the execution and command generation engines of StegBench, this function then generates a steganographic database with specified name and embedding ratio. The function takes the following parameters:
- <b>embeddor_set_uuid</b>(str): The UUID of the embeddor set that you wantn to use. 
- <b>database_uuid</b>(str): The UUID of the datatabase that you want to embed.
- <b>ratio</b>(float): The embedding ratio to use to generate the database. For spatial images, this is bits per pixel. For frequency images, this is bits per nonzero AC DC coefficient. 
- <b>name</b>(str): The name of the steganographic image that will be generated

In [18]:
stego_db_uuid = stegbench.embed(embeddor_set_uuid, TOY_database_uuid, 0.5, 'STEGANOGRAPHIC DATABASE')

generating commands...


0it [00:00, ?it/s]
  0%|          | 0/10 [00:00<?, ?it/s]

commands generated...
setting up embeddors...
completed.
embedding...


  0%|          | 0/2 [00:00<?, ?it/s]

completed.
processing embeding results...
completed.
terminating processes...


100%|██████████| 2/2 [00:00<00:00,  2.35it/s]

completed.
The UUID of the dataset you have created is: 90923375-9860-4be8-9c5e-0bf545d165dd


## Step 7. Steganographic Database Verification

To verify that a steganographic database has been generated correctly, we have provided the following function:

```python
stegbench.verify(stego_database_uuid)
```

The verification function extracts the message that was embedded inside the images of the steganographic database. It then verifies that the message extracted is the same as the message that was embedded. The function takes the following parameters:
- <b>stego_database_uuid</b>(str): UUID of a generated steganographic database.

In [19]:
stegbench.verify(stego_db_uuid)

0it [00:00, ?it/s]
  0%|          | 0/10 [00:00<?, ?it/s]

running pre commands
completed.
running commands


100%|██████████| 10/10 [00:04<00:00,  1.81s/it]
0it [00:00, ?it/s]
  0%|          | 0/12 [00:00<?, ?it/s]

completed.
running post commands.
completed.
terminating processes...


100%|██████████| 12/12 [00:00<00:00, 13.00it/s]

completed.
Listing all verification results...
----------------------------------------------------------------------------------------------------
Specific Embeddor Results
	Name: LSBSteg
	UUID: 76e9535b-eabd-4aa6-8adf-5edfc1bc8f2b
		Correctly Embedded (%): 100.0
		Incorrect Embedding (%): 0.0
	Name: SteganoLSB
	UUID: eb661d7e-a7c1-4db4-b754-70b70f1004ee
		Correctly Embedded (%): 100.0
		Incorrect Embedding (%): 0.0
----------------------------------------------------------------------------------------------------
Total Results
	Correctly Embedded (%): 100.0
	Incorrect Embedding (%): 0.0


## Step 8. Testing out StegDetectors

To analyze a steganographic database using the stegdetectors available in the system, we have provided the following function:

```python
stegbench.detect(detector_set_uuid, database_uuids)
```

The detect function takes a detector_set_uuid to retrieve a list of stegdetector tools. It then takes a list of database_uuids to retrieve a list of databases to test the stegdetector tools on. Once this is done, the execution and command generation engine use the configurations of each of the tools to test the database. The analyzer engine then reports the results. The function takes the following parameters:
- <b>detector_set_uuid</b>(str): This is the UUID of the detector set that you want to analyze the databases with. 
- <b>database_uuids</b>(list): This is a list of database UUIDs that we want to check the detectors with. 

In [20]:
database_uuids = [TOY_database_uuid, stego_db_uuid]
stegbench.detect(detector_set_uuid, database_uuids)

0it [00:00, ?it/s]
  0%|          | 0/11 [00:00<?, ?it/s]

collecting results from following databases: ['730515e3-572f-4d79-a53f-8049961aae0f', '90923375-9860-4be8-9c5e-0bf545d165dd']
setting up detectors...
completed.
analyzing images...


  0%|          | 0/1 [00:00<?, ?it/s]

completed.
processing image results...


100%|██████████| 12/12 [00:00<00:00, 533.62it/s]
0it [00:00, ?it/s]
  0%|          | 0/11 [00:00<?, ?it/s]

completed.
terminating processes...
completed.
setting up detectors...
completed.
analyzing images...


  0%|          | 0/1 [00:00<?, ?it/s]

completed.
processing image results...


100%|██████████| 12/12 [00:00<00:00, 436.49it/s]

completed.
terminating processes...
completed.
Experiment information
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Database Information
	UUID: 730515e3-572f-4d79-a53f-8049961aae0f
		Image Types: ['png']
----------------------------------------------------------------------------------------------------
Database Information
	UUID: 90923375-9860-4be8-9c5e-0bf545d165dd
		Source DB: 730515e3-572f-4d79-a53f-8049961aae0f
		Source Embeddor Set: 23fea9ab-7716-4d35-aa77-8dadf46f5dcd
		Image Types: ['png']
		Payload: 0.5
----------------------------------------------------------------------------------------------------
Embeddor Set Information
	UUID: 23fea9ab-7716-4d35-aa77-8dadf46f5dcd
		Compatible Types: {'png'}
		Embeddors: 2
			(LSBSteg, 76e9535b-eabd-4aa6-8adf-5edfc1bc8f2b)
			(SteganoLSB, eb661d7e-a7c1-4db4-b754-70b70f1004ee)
--------

## Step 9. Generating a Labeled Training Dataset

Since libraries like [StegDetect](http://github.com/DAI-Lab/StegDetect) require certain labeling formats for the data. We have provided functionality to generate these labels for your training datasets. In this case, let us take the toy dataset and its steganographic counterpart to generate the labeled data. The function we provide is the following:

```python
stegbench.generate_labels(stego_database_uuids, output_csv_file, relative=False)
```

This function takes three parameters that are:

- <b>stego_database_uuids</b>(list). List of uuids of steganographic databases
- <b>output_csv_file</b>(str): List of paths to directories that contain configuration files. Useful for batch processing. 
- <b>relative</b>(bool): If true, the file paths put into the csv file will be relative paths. Otherwise, they will be absolute paths. 

Once this file is generated, you can use this labels file to train other models based of the images that you have generated using StegBench. 

In [21]:
stegbench.generate_labels([stego_db_uuid], 'labels.csv', relative=True)

The labels file can be found here: /Users/anene/Documents/MEng/Steganography/Evaluator/stegbench/notebooks/stegbench_asssets/db/labels.csv
